In [1]:
#import dependencies
import tensorflow as tf
import numpy as np
import os

In [ ]:
# Import text
# text = path

In [ ]:
#Get unique caracters in text
vocab = sorted(set(text))

In [ ]:
#Create character to index conversion and back, and assign index values to text     
char_to_idx = {u:i for i, u in enumerate(vocab)}
idx_to_char = np.array(vocab)
text_as_int = np.array([char_to_idx[char] for char in text])

In [ ]:
#Create dataset batches
dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 200
examples_per_epoch = len(text)//(seq_length+1)
sequences = dataset.batch(seq_length+1, drop_remainder = True)

In [ ]:
def split_input_target(chunk):
  input = chunk[:-1]
  target = chunk[1:]
  return input, target

data = sequences.map(split_input_target)

In [ ]:
buffer_size = 10000
batch_size = 4
data = data.shuffle(buffer_size).batch(batch_size, drop_remainder=True)